## 🧪 E4: 데이터 증강 실험

> **목표**: 데이터 증강으로 일반 욕설 탐지 강화 + 특수문자 오탐 감소

### 배경
- 변형 욕설에 집중하다 보니 **일반 욕설 탐지율이 낮음**
- 특수문자(`@`, `^`, `*` 등)를 **욕설로 오판**하는 경향

### 실험 내용
1. **LOL 욕설 데이터** 추가 (악성 라벨) → 일반/변형 욕설 탐지 강화
2. **특수문자 정상 데이터** 추가 (정상 라벨) → FP 감소

### 변경 사항
- 데이터: 기존 40,000개 + LOL 욕설 + 특수문자 정상
- `metric_for_best_model`: f1 (기존 유지)
- `class_weights`: 기본 (1:1) - E5에서 조정 예정
- `threshold`: 0.5 (기존 유지) - 나중에 0.7로 조정 예정

In [1]:
# ========================================
# 📦 패키지 설치 (필요시)
# ========================================
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
%pip install transformers==4.42.0 datasets accelerate scikit-learn pandas matplotlib seaborn -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# ========================================
# 🔧 경로 설정
# ========================================
import os

DATA_DIR = r"c:\00_Sookmyunguniv\DACOS(2025)\2학기\프로젝트\github\25-2-team3\data"
OUTPUT_DIR = r"./E4_output"

# 데이터 파일 경로
MAIN_DATA = os.path.join(DATA_DIR, "combined_abusive_shuffled_20k.csv")
NORMAL_DATA = os.path.join(DATA_DIR, "nonabusive_merged_shuffled_sample20000.csv")
LOL_DATA = os.path.join(DATA_DIR, "LOL_badwords_all_merged.csv")

if not os.path.exists(DATA_DIR):
    print(f"⚠️ 데이터 폴더를 찾을 수 없습니다: {DATA_DIR}")
else:
    print(f"📂 데이터 폴더: {DATA_DIR}")
print(f"📁 출력 폴더: {OUTPUT_DIR}")

📂 데이터 폴더: c:\00_Sookmyunguniv\DACOS(2025)\2학기\프로젝트\github\25-2-team3\data
📁 출력 폴더: ./E4_output


In [3]:
# ========================================
# 📚 라이브러리 임포트
# ========================================
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score
)
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import matplotlib.pyplot as plt
import seaborn as sns

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.is_available():
    print(f"✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU 사용 불가, CPU로 실행")

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

✅ GPU 사용 가능: NVIDIA GeForce RTX 4060 Ti


In [4]:
# ========================================
# 📊 1) 기존 데이터 로드
# ========================================
df_main = pd.read_csv(MAIN_DATA)
df_normal = pd.read_csv(NORMAL_DATA)

# 기존 데이터 합치기 (text, label 컬럼 통일)
if 'text' not in df_main.columns:
    df_main = df_main.rename(columns={df_main.columns[0]: 'text'})
if 'label' not in df_main.columns:
    df_main['label'] = 1  # 악성

if 'text' not in df_normal.columns:
    df_normal = df_normal.rename(columns={df_normal.columns[0]: 'text'})
if 'label' not in df_normal.columns:
    df_normal['label'] = 0  # 정상

df_base = pd.concat([df_main[['text', 'label']], df_normal[['text', 'label']]], ignore_index=True)
print(f"📊 기존 데이터: {len(df_base)}개")
print(f"   - 정상: {len(df_base[df_base['label']==0])}개")
print(f"   - 악성: {len(df_base[df_base['label']==1])}개")

📊 기존 데이터: 40000개
   - 정상: 22882개
   - 악성: 17118개


In [5]:
# ========================================
# 🎮 2) LOL 욕설 데이터 추가
# ========================================
df_lol = pd.read_csv(LOL_DATA)
print(f"📊 LOL 욕설 원본: {len(df_lol)}개 행")

# origin(원본 욕설) + variant(변형 욕설) 모두 사용
lol_origins = df_lol['origin'].dropna().unique().tolist()
lol_variants = df_lol['variant'].dropna().unique().tolist()

# 합쳐서 중복 제거
all_lol_words = list(set(lol_origins + lol_variants))
print(f"   - origin(원본): {len(lol_origins)}개")
print(f"   - variant(변형): {len(lol_variants)}개")
print(f"   - 합계(중복제거): {len(all_lol_words)}개")

df_lol_aug = pd.DataFrame({
    'text': all_lol_words,
    'label': 1  # 악성
})

# 기존 데이터와 중복 제거
existing_texts = set(df_base['text'].astype(str).str.lower())
df_lol_aug = df_lol_aug[~df_lol_aug['text'].astype(str).str.lower().isin(existing_texts)]

print(f"✅ LOL 욕설 추가: {len(df_lol_aug)}개 (기존 데이터와 중복 제거 후)")

📊 LOL 욕설 원본: 13504개 행
   - origin(원본): 441개
   - variant(변형): 13304개
   - 합계(중복제거): 13532개
✅ LOL 욕설 추가: 12777개 (기존 데이터와 중복 제거 후)


In [6]:
# ========================================
# 🔣 3) 특수문자 정상 데이터 생성
# ========================================
# 특수문자만 있거나 특수문자 중심의 정상 문장들
special_char_samples = [
    # 단일 특수문자
    "@", "#", "$", "%", "^", "&", "*", "!", "?", "~",
    ".", ",", ";", ":", "'", "\"", "`",
    "(", ")", "[", "]", "{", "}", "<", ">",
    "/", "\\", "|", "-", "_", "=", "+",
    
    # 이모티콘/표정
    "^^", "^0^", "^_^", "^^;", "^o^", "^^7","^-^",
    ":)", ":(", ":D", ":P", ";)", ":/",
    "ㅋ", "ㅎ", "ㅠ", "ㅜ", "ㄷ", "ㄱ",
    "ㅋㅋ", "ㅎㅎ", "ㅠㅠ", "ㅜㅜ", "ㄷㄷ", "ㄱㄱ",
    "ㅋㅋㅋ", "ㅎㅎㅎ", "ㅠㅠㅠ", "ㅜㅜㅜ",
    "ㅋㅋㅋㅋㅋ", "ㅎㅎㅎㅎㅎ", "ㅠㅠㅠㅠㅠ",
    "ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ",
    
    # 특수문자 조합
    "@@@", "###", "***", "!!!", "???", "~~~",
    "...", "....", ".....",
    "!?", "?!", "!?!", "?!?",
    "ㅋㅎ", "ㅎㅋ", "ㅋㅋㅎㅎ",
    
    # 띄어쓰기 포함
    "^ ^", "^ _ ^", ". . .", "? ? ?",
    "ㅋ ㅋ ㅋ", "ㅎ ㅎ ㅎ",
    
    # 멘션/태그 형식
    "@user", "@님", "#태그", "#해시태그",
    "@all", "@everyone", "@here"
    
    # 숫자+특수문자
    "1234", "12345", "123456", "1", "2", "3",
    "100", "200", "300", "1000", "10000",
    "1!", "2@", "3#", "4$", "5%",
    
    # 짧은 정상 문자열 (오탐 가능성 높은 것들)
    "ㅇㅇ", "ㅇㅋ", "ㄴㄴ", "ㄱㄱ", "ㄴㅇ", "ㅇㄴ",
    "ㅎㅇ", "ㅂㅂ", "ㄱㅅ", "ㅊㅋ", "ㄳ",
    "ㅇㅇㅇ", "ㄴㄴㄴ", "ㄱㄱㄱ",
    "ok", "OK", "ㅇㅋ", "오키", "오케이",
    "ㄱㄱㄱㄱ", "고고고고", "고고", "ㄱㄱ",
    
    # 게임/인터넷 은어 (정상)
    "gg", "GG", "wp", "WP", "glhf", "GLHF",
    "lol", "LOL", "lmao", "LMAO",
    "ㄹㅇ", "ㅇㅈ"
]

df_special = pd.DataFrame({
    'text': special_char_samples,
    'label': 0  # 정상
})

# 중복 제거
df_special = df_special[~df_special['text'].str.lower().isin(existing_texts)]

print(f"✅ 특수문자 정상 데이터 추가: {len(df_special)}개")

✅ 특수문자 정상 데이터 추가: 143개


In [7]:
# ========================================
# 📊 4) 데이터 합치기
# ========================================
df = pd.concat([df_base, df_lol_aug, df_special], ignore_index=True)

# 셔플
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n📊 최종 데이터 분포:")
print(f"   총 데이터: {len(df)}개")
print(f"   - 정상(0): {len(df[df['label']==0])}개")
print(f"   - 악성(1): {len(df[df['label']==1])}개")
print(f"   - 비율: 1:{len(df[df['label']==0])/len(df[df['label']==1]):.2f}")

print(f"\n📈 데이터 증가:")
print(f"   기존: {len(df_base)}개")
print(f"   LOL 욕설 추가: +{len(df_lol_aug)}개")
print(f"   특수문자 정상 추가: +{len(df_special)}개")
print(f"   최종: {len(df)}개")


📊 최종 데이터 분포:
   총 데이터: 52920개
   - 정상(0): 23025개
   - 악성(1): 29895개
   - 비율: 1:0.77

📈 데이터 증가:
   기존: 40000개
   LOL 욕설 추가: +12777개
   특수문자 정상 추가: +143개
   최종: 52920개


In [8]:
# ========================================
# 🔀 5) Train/Test 분할
# ========================================
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label']
)
print(f"Train: {len(train_df)}, Test: {len(test_df)}")
print(f"Train 비율 - 정상: {len(train_df[train_df['label']==0])}, 악성: {len(train_df[train_df['label']==1])}")

Train: 42336, Test: 10584
Train 비율 - 정상: 18420, 악성: 23916


In [9]:
# ========================================
# 🔤 6) 토큰화
# ========================================
MODEL_ID = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

raw_ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(test_df),
})

tokenized = raw_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format("torch")

print("✅ 토큰화 완료!")

Map:   0%|          | 0/42336 [00:00<?, ? examples/s]

Map:   0%|          | 0/10584 [00:00<?, ? examples/s]

✅ 토큰화 완료!


In [10]:
# ========================================
# 🏋️ 7) 모델 및 Trainer 설정
# ========================================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, zero_division=0)
    rec = recall_score(labels, preds, zero_division=0)
    f1 = f1_score(labels, preds, zero_division=0)
    
    cm = confusion_matrix(labels, preds)
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    try:
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
        auc = roc_auc_score(labels, probs)
    except:
        auc = float("nan")
    
    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": auc,
        "FP": fp,
        "FN": fn,
        "FPR": fpr,
    }

base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=2)

os.makedirs(OUTPUT_DIR, exist_ok=True)

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = Trainer(
    model=base_model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("✅ Trainer 설정 완료!")
print("📌 E4 특징: LOL 욕설 + 특수문자 정상 데이터 증강")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Trainer 설정 완료!
📌 E4 특징: LOL 욕설 + 특수문자 정상 데이터 증강


In [11]:
# ========================================
# 🚀 8) 학습 실행
# ========================================
trainer.train()

  0%|          | 0/3969 [00:00<?, ?it/s]

{'loss': 0.6971, 'grad_norm': 1.899768590927124, 'learning_rate': 3.6272040302267e-06, 'epoch': 0.04}
{'loss': 0.5835, 'grad_norm': 1.7394795417785645, 'learning_rate': 7.405541561712847e-06, 'epoch': 0.08}
{'loss': 0.3738, 'grad_norm': 1.6148173809051514, 'learning_rate': 1.1183879093198994e-05, 'epoch': 0.11}
{'loss': 0.2779, 'grad_norm': 3.1240575313568115, 'learning_rate': 1.496221662468514e-05, 'epoch': 0.15}
{'loss': 0.2409, 'grad_norm': 2.0674993991851807, 'learning_rate': 1.8740554156171285e-05, 'epoch': 0.19}
{'loss': 0.216, 'grad_norm': 3.47833251953125, 'learning_rate': 2.244332493702771e-05, 'epoch': 0.23}
{'loss': 0.2161, 'grad_norm': 7.474817752838135, 'learning_rate': 2.6221662468513855e-05, 'epoch': 0.26}
{'loss': 0.2055, 'grad_norm': 7.625387668609619, 'learning_rate': 3e-05, 'epoch': 0.3}
{'loss': 0.1901, 'grad_norm': 3.4509928226470947, 'learning_rate': 2.958006718924972e-05, 'epoch': 0.34}
{'loss': 0.1762, 'grad_norm': 15.50816822052002, 'learning_rate': 2.916013437

  0%|          | 0/331 [00:00<?, ?it/s]

{'eval_loss': 0.145191490650177, 'eval_accuracy': 0.9406651549508692, 'eval_precision': 0.9661962014288203, 'eval_recall': 0.9274126108044823, 'eval_f1': 0.9464072367298174, 'eval_roc_auc': 0.9876297951262281, 'eval_FP': 194, 'eval_FN': 434, 'eval_FPR': 0.04212812160694897, 'eval_runtime': 12.391, 'eval_samples_per_second': 854.17, 'eval_steps_per_second': 26.713, 'epoch': 1.0}
{'loss': 0.1486, 'grad_norm': 4.415217399597168, 'learning_rate': 2.2029675251959686e-05, 'epoch': 1.02}
{'loss': 0.125, 'grad_norm': 3.174283742904663, 'learning_rate': 2.1609742441209405e-05, 'epoch': 1.06}
{'loss': 0.1147, 'grad_norm': 3.140873908996582, 'learning_rate': 2.1189809630459128e-05, 'epoch': 1.1}
{'loss': 0.1027, 'grad_norm': 1.679313063621521, 'learning_rate': 2.0769876819708847e-05, 'epoch': 1.13}
{'loss': 0.1151, 'grad_norm': 0.9049972891807556, 'learning_rate': 2.0349944008958566e-05, 'epoch': 1.17}
{'loss': 0.1098, 'grad_norm': 5.2293171882629395, 'learning_rate': 1.993001119820829e-05, 'epoc

  0%|          | 0/331 [00:00<?, ?it/s]

{'eval_loss': 0.1526689976453781, 'eval_accuracy': 0.9441609977324263, 'eval_precision': 0.9538409703504043, 'eval_recall': 0.9469811005184814, 'eval_f1': 0.9503986571548468, 'eval_roc_auc': 0.9889895125156652, 'eval_FP': 274, 'eval_FN': 317, 'eval_FPR': 0.05950054288816504, 'eval_runtime': 12.4206, 'eval_samples_per_second': 852.136, 'eval_steps_per_second': 26.649, 'epoch': 2.0}
{'loss': 0.087, 'grad_norm': 2.2846498489379883, 'learning_rate': 1.1119820828667414e-05, 'epoch': 2.0}
{'loss': 0.0434, 'grad_norm': 1.298702597618103, 'learning_rate': 1.0699888017917135e-05, 'epoch': 2.04}
{'loss': 0.0478, 'grad_norm': 1.679762840270996, 'learning_rate': 1.0279955207166852e-05, 'epoch': 2.08}
{'loss': 0.047, 'grad_norm': 2.8655145168304443, 'learning_rate': 9.860022396416573e-06, 'epoch': 2.12}
{'loss': 0.0456, 'grad_norm': 2.778520345687866, 'learning_rate': 9.440089585666292e-06, 'epoch': 2.15}
{'loss': 0.0527, 'grad_norm': 0.8934487700462341, 'learning_rate': 9.020156774916013e-06, 'epo

  0%|          | 0/331 [00:00<?, ?it/s]

{'eval_loss': 0.20777806639671326, 'eval_accuracy': 0.9453892668178382, 'eval_precision': 0.9581000848176421, 'eval_recall': 0.944639571834755, 'eval_f1': 0.9513222166077143, 'eval_roc_auc': 0.988817502590954, 'eval_FP': 247, 'eval_FN': 331, 'eval_FPR': 0.053637350705754615, 'eval_runtime': 12.2061, 'eval_samples_per_second': 867.11, 'eval_steps_per_second': 27.118, 'epoch': 3.0}
{'train_runtime': 3431.6155, 'train_samples_per_second': 37.011, 'train_steps_per_second': 1.157, 'train_loss': 0.12629208514599694, 'epoch': 3.0}


TrainOutput(global_step=3969, training_loss=0.12629208514599694, metrics={'train_runtime': 3431.6155, 'train_samples_per_second': 37.011, 'train_steps_per_second': 1.157, 'total_flos': 8354302229790720.0, 'train_loss': 0.12629208514599694, 'epoch': 3.0})

In [12]:
# ========================================
# 📊 9) 최종 평가
# ========================================
final_metrics = trainer.evaluate()

print("\n" + "="*60)
print("📊 E4 최종 평가 결과 (데이터 증강)")
print("="*60)
for key, value in final_metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

  0%|          | 0/331 [00:00<?, ?it/s]


📊 E4 최종 평가 결과 (데이터 증강)
  eval_loss: 0.2078
  eval_accuracy: 0.9454
  eval_precision: 0.9581
  eval_recall: 0.9446
  eval_f1: 0.9513
  eval_roc_auc: 0.9888
  eval_FP: 247
  eval_FN: 331
  eval_FPR: 0.0536
  eval_runtime: 12.1898
  eval_samples_per_second: 868.2680
  eval_steps_per_second: 27.1540
  epoch: 3.0000


In [13]:
# ========================================
# 📝 10) 실험 결과 기록
# ========================================
print("\n" + "="*60)
print("📝 실험 결과 기록 (복사용)")
print("="*60)

# 변수 미리 포맷팅
fpr_str = f"{final_metrics['eval_FPR']:.4f}" if final_metrics.get('eval_FPR') is not None else 'N/A'
prec_str = f"{final_metrics['eval_precision']:.4f}" if final_metrics.get('eval_precision') is not None else 'N/A'
rec_str = f"{final_metrics['eval_recall']:.4f}" if final_metrics.get('eval_recall') is not None else 'N/A'
f1_str = f"{final_metrics['eval_f1']:.4f}" if final_metrics.get('eval_f1') is not None else 'N/A'
auc_str = f"{final_metrics['eval_roc_auc']:.4f}" if final_metrics.get('eval_roc_auc') is not None else 'N/A'

print(f"""
#### 실험 ID: E4
- 변경 사항:
  - 데이터 증강: LOL 욕설({len(df_lol_aug)}개) + 특수문자 정상({len(df_special)}개)
  - 총 데이터: {len(df)}개 (기존 {len(df_base)}개 → +{len(df)-len(df_base)}개)
  - metric_for_best_model: f1
  - class_weights: 기본 (1:1)
- 데이터:
  - train/validation split: 0.8/0.2 (random_state=42, stratify=label)
  - max_length=128, batch_size=32, epochs=3
- Validation 성능:
  - FP(정상→악성): {final_metrics.get('eval_FP', 'N/A')}
  - FN(악성→정상): {final_metrics.get('eval_FN', 'N/A')}
  - FPR: {fpr_str}
  - Precision: {prec_str}
  - Recall: {rec_str}
  - F1: {f1_str}
  - AUC: {auc_str}
- 코멘트:
  - LOL 욕설 데이터로 일반/변형 욕설 탐지 강화 예상
  - 특수문자 정상 데이터로 FP 감소 예상
""")


📝 실험 결과 기록 (복사용)

#### 실험 ID: E4
- 변경 사항:
  - 데이터 증강: LOL 욕설(12777개) + 특수문자 정상(143개)
  - 총 데이터: 52920개 (기존 40000개 → +12920개)
  - metric_for_best_model: f1
  - class_weights: 기본 (1:1)
- 데이터:
  - train/validation split: 0.8/0.2 (random_state=42, stratify=label)
  - max_length=128, batch_size=32, epochs=3
- Validation 성능:
  - FP(정상→악성): 247
  - FN(악성→정상): 331
  - FPR: 0.0536
  - Precision: 0.9581
  - Recall: 0.9446
  - F1: 0.9513
  - AUC: 0.9888
- 코멘트:
  - LOL 욕설 데이터로 일반/변형 욕설 탐지 강화 예상
  - 특수문자 정상 데이터로 FP 감소 예상



In [14]:
# ========================================
# 💾 11) 모델 저장
# ========================================
SAVE_DIR = os.path.join(OUTPUT_DIR, "best_model")
os.makedirs(SAVE_DIR, exist_ok=True)
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print(f"✅ 모델 저장 완료: {SAVE_DIR}")

✅ 모델 저장 완료: ./E4_output\best_model


---
## 📋 다음 단계

E4 결과가 좋으면:
1. **E5**: E4 기반 + 가중치 반영 (욕설:정상 = 1:5)
2. **Threshold 튜닝**: 0.7 기준으로 조정

E4 결과가 안 좋으면:
- 데이터 비율 조정
- LOL 욕설 샘플링 비율 조정